# **Building a Question Answering Pipeline Using LangGraph and OpenAI LLM**

In [1]:
# Import required components from LangGraph and LangChain-OpenAI
from langgraph.graph import StateGraph, START, END  # Workflow graph and entry/exit points
from langchain_openai import ChatOpenAI             # LLM wrapper for OpenAI models
from typing import TypedDict                        # For strong type annotation of state objects
from dotenv import load_dotenv                      # To read OpenAI (or other) keys from a .env file

# Load environment variables (OPENAI_API_KEY) from a .env file.
load_dotenv()

True

In [2]:
# Instantiate the OpenAI language model. Here, we specify the model as "gpt-5".
model = ChatOpenAI(model="gpt-5")

- In LangGraph, the state is a Python dictionary (often explicitly typed using TypedDict) that holds the data as it moves through the workflow. Each node (function) in the StateGraph reads from and updates this state, passing it along to the next step. 

- This allows information like inputs, intermediate computations, and outputs to be shared and modified as your workflow progresses. 

- In short, the state is the single source of truth representing your workflow's current data at any point in the graph.

In [3]:
# Define the structure of the state as it flows through the graph.
class LLMState(TypedDict):
    question: str   # The input question
    answer: str     # The model's answer (to be filled in by our node)

In [4]:
# Define a node function: given a state with a 'question', it generates an 'answer' using the LLM.
def llm_qa(state: LLMState) -> LLMState:
    question = state['question']  # Extract the question
    
    # Format the input prompt for the language model
    prompt = f"Answer the following question {question}"
    
    # Invoke the OpenAI model, extracting the answer from the response
    answer = model.invoke(prompt).content

    # Store the answer back in the state
    state['answer'] = answer

    return state  # Return the updated state

In [5]:
# Build the workflow graph using our state structure.
graph = StateGraph(LLMState)

# Add our LLM node to the graph; this is the only computational step in this pipeline.
graph.add_node('llm_qa', llm_qa)

# Define the workflow edges (execution order).
graph.add_edge(START, 'llm_qa')      # Start → llm_qa
graph.add_edge('llm_qa', END)        # llm_qa → End

# Compile the workflow so that it's ready to run.
workflow = graph.compile()

In [6]:
# Prepare the initial state with the question we want answered.
initial_state = {'question': 'What is the Future of Agentic AI?'}

# Invoke (execute) the graph pipeline with our initial state.
final_state = workflow.invoke(initial_state)

In [7]:

# Print out the answer provided by the LLM.
print(final_state['answer'])

Short answer: Agentic AI is moving from chatbots to reliable, goal‑directed software that can plan, use tools, coordinate with other agents, and act on your behalf under guardrails. In the near term it will automate many digital workflows with human oversight; over the medium term it will coordinate complex projects and increasingly control physical systems. The bottlenecks are reliability, safety, and governance—not raw capability.

Key shifts to expect
- From copilots to co‑workers: Agents will run multi‑step tasks end‑to‑end (research → decision support → execution → reporting), pausing for approval only at risk boundaries.
- Stronger reasoning and planning: Architectures that separate planning from acting, search‑augmented decoding, verifiable tool use, and program‑of‑thought methods will reduce errors on long‑horizon tasks.
- Tool use as a first‑class capability: Standardized tool interfaces (e.g., function calling, model context protocols) will let agents compose software, data, 